In [ ]:
import boto3
import json
session = boto3.Session(profile_name="dev_admin")
# Initialize a boto3 client
elbv2 = session.client('elbv2')

def list_rules(listener_arn):
    """List all rules for a given listener ARN."""
    response = elbv2.describe_rules(ListenerArn=listener_arn)
    return response['Rules']

def backup_rules(listener_arn, backup_file_path):
    """Backup all rules for a given listener ARN to a specified file."""
    rules = list_rules(listener_arn)  # Assuming list_rules is the function you provided
    # Simplify the rules for backup purposes
    simplified_rules = []
    for rule in rules:
        simplified_rule = {
            'Priority': rule['Priority'],
            'Conditions': rule['Conditions'],
            'Actions': rule['Actions']
        }
        if rule['Priority'] != 'default':
            simplified_rules.append(simplified_rule)
    
    with open(backup_file_path, 'w') as file:
        json.dump(simplified_rules, file, indent=4)
        
    print(f"Backup completed for listener {listener_arn}.")

def restore_rules(listener_arn, backup_file_path):
    """Restore rules for a given listener ARN from a specified backup file."""
    try:
        with open(backup_file_path, 'r') as file:
            rules = json.load(file)
        
        for rule in rules:
            response = elbv2.create_rule(
                ListenerArn=listener_arn,
                Conditions=rule['Conditions'],
                Priority=int(rule['Priority']),
                Actions=rule['Actions']
            )
            print(f"Rule with priority {rule['Priority']} restored.")
            
    except FileNotFoundError:
        print("Backup file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


 
def modify_rule_priorities(rules):
    """Calculate new priorities and prepare modifications."""
    modifications = []
    for rule in rules:
        if 'Priority' in rule and rule['Priority'].isdigit():  # Skip the default rule
            new_priority = int(rule['Priority']) + 100
            modifications.append({
                'RuleArn': rule['RuleArn'],
                'Priority': int(new_priority)
            })
    return modifications

def update_rule_priorities(modifications):
    """Update rules with the new priorities."""
    if modifications:
        elbv2.set_rule_priorities(RulePriorities=modifications)


listener_arn = ''

rules = list_rules(listener_arn)
print(rules)
backup_file_path = "dev-ssl-rules.json"
backup_rules(listener_arn, backup_file_path)
# restore_rules(listener_arn, backup_file_path)

In [ ]:
modifications = modify_rule_priorities(rules)

update_rule_priorities(modifications)
rules = list_rules(listener_arn)
print("Rule priorities have been updated.")
print(rules)

In [23]:
import boto3
import json
def get_ssl_listener_arn(alb_arn,session):
    # Create an ELBv2 client
    
    client = session.client('elbv2')

    # Retrieve all listeners for the specified ALB ARN
    try:
        listeners = client.describe_listeners(LoadBalancerArn=alb_arn)['Listeners']
    except client.exceptions.LoadBalancerNotFoundException:
        print(f"The specified ALB ARN '{alb_arn}' does not exist.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    # Iterate over the listeners to find an SSL/TLS listener
    for listener in listeners:
        if listener['Protocol'] in ('HTTPS', 'TLS'):
            return listener['ListenerArn']

    # If no SSL listener is found
    print("No SSL listener found for the specified ALB ARN.")
    return None


def list_rules(listener_arn,session):
    """List all rules for a given listener ARN."""
    elbv2 = session.client('elbv2')
    response = elbv2.describe_rules(ListenerArn=listener_arn)
    return response['Rules']
def backup_rules(listener_arn, backup_file_path,session):
    """Backup all rules for a given listener ARN to a specified file."""
    rules = list_rules(listener_arn,session)  # Assuming list_rules is the function you provided
    # Simplify the rules for backup purposes
    simplified_rules = []
    for rule in rules:
        simplified_rule = {
            'Priority': rule['Priority'],
            'Conditions': rule['Conditions'],
            'Actions': rule['Actions']
        }
        if rule['Priority'] != 'default':
            simplified_rules.append(simplified_rule)
    
    with open(backup_file_path, 'w') as file:
        json.dump(simplified_rules, file, indent=4)
        
    print(f"Backup completed for listener {listener_arn}.")

In [ ]:



alb_arn = ''
session = boto3.Session(profile_name="dev_admin")

ssl_listener_arn = get_ssl_listener_arn(alb_arn,session)
if ssl_listener_arn:
    print(f"SSL Listener ARN: {ssl_listener_arn}")
else:
    print("Could not find an SSL Listener for the specified ALB.")

backup_file_path = "dev-ssl-rules.json"
backup_rules(ssl_listener_arn, backup_file_path,session)